In [1]:
import os 

In [2]:
%pwd

'c:\\Users\\Lenovo\\Downloads\\ml-model-tracking-with-mlflow\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Lenovo\\Downloads\\ml-model-tracking-with-mlflow'

In [12]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [13]:
from src.mlProject_mlops.constants import *
from src.mlProject_mlops.utils.common import *

In [14]:
class configrationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)  
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) ->DataIngestionConfig:
        config=self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [15]:
import os
import urllib.request as request
import zipfile
from src.mlProject_mlops.logging import logger
from src.mlProject_mlops.utils.common import *


In [18]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config


    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers=request.urlretrieve(
                url=self.config.source_url,
                filename=self.config.local_data_file
            )

            logger.info(f"{filename} download with the following info: \n{headers}")
        else:
            logger.info(f"File already exists of zie:{get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [19]:
try:
    config=configrationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-07-28 01:27:38,542: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-28 01:27:38,552: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-28 01:27:38,559: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-07-28 01:27:38,559: INFO: common: created directoris at:artifacts]
[2025-07-28 01:27:38,568: INFO: common: created directoris at:artifacts/data_ingestion]
[2025-07-28 01:27:39,658: INFO: 4002691618: artifacts/data_ingestion/data.zip download with the following info: 
Connection: close
Content-Length: 25166
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "8ad93de31983f25c2dd5854f911690731827e61c62490665e723fe952844fc5e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: B840:3B0B4A:2754D7:6412AF:688684B3
Accept-Ranges: bytes
Date: Sun,